In [1]:
import sys
sys.path.insert(0, "/chia-utils")
from IPython.display import JSON

from chia.types.condition_opcodes import ConditionOpcode
from chia.types.blockchain_format.program import Program
from chia.types.spend_bundle import SpendBundle

from clvm_tools.binutils import disassemble

import sim
import standard_transaction

In [2]:
# create sim network, alice's and bob's wallets
network = await sim.get_network_async()
alice = network.make_wallet("alice")
bob = network.make_wallet("bob")

In [3]:
# alice farms the block and have 2 XCH
await network.farm_block(farmer=alice)
alice.balance()

2000000000000

In [4]:
# bob still has 0 mojos
bob.balance()

0

In [5]:
# retrieve one of alice's coin
coin_records = await network.sim_client.get_coin_records_by_puzzle_hash(alice.puzzle_hash)
amount = 1_000_000_000_000
coin = next(cr.coin for cr in coin_records if cr.spent == False and cr.coin.amount > amount)
JSON(coin.to_json_dict())

<IPython.core.display.JSON object>

In [6]:
# create coin for bob and change back to alice.
conditions = [
    Program.to(
            [
                ConditionOpcode.CREATE_COIN,
                bob.puzzle_hash,
                amount,
            ]
        ),
    Program.to(
            [
                ConditionOpcode.CREATE_COIN,
                alice.puzzle_hash,
                coin.amount - amount,
            ]
        )
]
disassemble(conditions[0])

'(51 0x87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5 0x00e8d4a51000)'

In [7]:
coin_spend, sig = standard_transaction.get_normal_coin_spend_and_sig(alice.sk_, coin, conditions)
JSON(coin_spend.to_json_dict())

<IPython.core.display.JSON object>

In [8]:
sig

<G2Element a0a3b5d7540c1fa3deb60d9399e3e641be0ad556f47a2c394048387e985c11969e16062b993a888dcf039ace1cdbfded18288715843e91cf2fe9e1a22b46be3dc5e57a66678d234391ac34d47ef473fb9c038e308b024161fb12eeac403a836d>

In [9]:
spend_bundle = SpendBundle([coin_spend], sig)
JSON(spend_bundle.to_json_dict())

<IPython.core.display.JSON object>

In [10]:
result = await network.push_tx(spend_bundle)
result

{'additions': [Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: 87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5>, amount=1000000000000),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=750000000000)],
 'removals': [Coin(parent_coin_info=<bytes32: e3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1750000000000)]}

In [11]:
alice.balance()

1000000000000

In [12]:
bob.balance()

1000000000000